# Create calibration dataset for DNNDK

In [21]:
import tensorflow as tf

In [22]:
import os
import shutil
import glob

In [23]:
import random

In [24]:
from tqdm import tqdm

In [25]:
print("TensorFlow version: %s" % tf.__version__)

TensorFlow version: 1.14.0


***

- Settings

In [26]:
dir_train = "D:\\dataset\\imagenet\\train"

In [27]:
output_dir = '_calib'

In [28]:
image_list = 'calib.txt'

In [29]:
img_size = 224

- List calibration images

In [30]:
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.mkdir(output_dir)

In [31]:
# Enumerate sub directories
dirs = glob.glob(os.path.join(dir_train, '**', ''), recursive=False)

In [32]:
calib_imgs = list()
for d in dirs:
    imgs = glob.glob(os.path.join(d, '*.JPEG'))
    calib_imgs.append( imgs[int(random.uniform(0, len(imgs) - 1))] )

- Save to destination folder after resizing

In [33]:
tf.enable_eager_execution()

In [34]:
def calc_new_size(img, smallest_side):
    height = tf.cast(img.shape[0], dtype=tf.float32)  # Height
    width =  tf.cast(img.shape[1], dtype=tf.float32)  # Width
    smallest_side = tf.cast(smallest_side, dtype=tf.float32)
    scale = tf.cond(tf.greater(height, width),
                  lambda: smallest_side / width,
                  lambda: smallest_side / height)

    new_height = tf.multiply(height, scale)
    new_width = tf.multiply(width, scale)

    offset_h = tf.cast(tf.subtract(new_height, smallest_side) / 2, tf.int32)
    offset_w = tf.cast(tf.subtract(new_width, smallest_side) / 2, tf.int32)
    
    new_height = tf.cast(new_height, tf.int32)
    new_width  = tf.cast(new_width, tf.int32)
    
    return new_height, new_width, offset_h, offset_w

In [35]:
bar = tqdm(total = len(calib_imgs))
bar.set_description('Progress')

for img_path in calib_imgs:
    with tf.gfile.GFile(img_path, 'rb') as fr:
        img = tf.image.decode_image(fr.read())

        # Calculate Offset & size after resize
        new_height, new_width, offset_h, offset_w = calc_new_size(img, img_size + 1)

        # expand_dims(): Input to resize_bilinear must be a 4-D tensor
        img_resize = tf.image.resize_bilinear(
            tf.expand_dims(img, 0), [new_height, new_width], align_corners=False)

        # Crop
        img2 = tf.image.crop_to_bounding_box(
            img_resize, offset_h, offset_w, img_size, img_size)

        # Save as .png
        _, name = os.path.split(img_path)
        name = os.path.splitext(name)[0]

        png_data = tf.image.encode_png(tf.cast(img2[0], dtype=tf.uint8))
        with tf.gfile.GFile(os.path.join(output_dir, name + '.png'), 'wb') as fw:
            fw.write(png_data.numpy())
        
    # Update progress bar
    bar.update(1)

Progress: 100%|████████████████████████████████████████████████████████████████████| 1000/1000 [02:21<00:00,  6.58it/s]

In [36]:
with open(os.path.join(output_dir, image_list), 'w') as f_list:
    # Save list of images
    _, name = os.path.split(img_path)
    name = os.path.splitext(name)[0]
    f_list.write(output_dir+'/'+name+'.png'+'\n')